In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split,GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import warnings  
warnings.filterwarnings('ignore')

from sklearn.metrics import f1_score

from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier,XGBRFClassifier
from sklearn.model_selection import cross_val_score

from collections import Counter

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve


In [ ]:
train=pd.read_csv('/kaggle/input/data-storm-20/Hotel-A-train.csv')
vali=pd.read_csv('/kaggle/input/data-storm-20/Hotel-A-validation.csv')

test=pd.read_csv('/kaggle/input/data-storm-20/Hotel-A-test.csv')
test_set=test

In [ ]:
vali

In [ ]:
train.dtypes=='int64'

In [ ]:
new_I=[]

for i in train['Income']:
    
    if (i=='<25K'):
        new_I.append(1)
    elif (i=='25K --50K'):
        new_I.append(2)
    elif (i=='50K -- 100K'):
        new_I.append(3)
    else:
        new_I.append(4)
new_I=pd.DataFrame(new_I)
train.insert(6,'New_Income',new_I)
    
new_I=[]
for i in vali['Income']:
    
    if (i=='<25K'):
        new_I.append(1)
    elif (i=='25K --50K'):
        new_I.append(2)
    elif (i=='50K -- 100K'):
        new_I.append(3)
    else:
        new_I.append(4)
new_I=pd.DataFrame(new_I)
vali.insert(6,'New_Income',new_I)

new_I=[]
for i in test['Income']:
    
    if (i=='<25K'):
        new_I.append(1)
    elif (i=='25K --50K'):
        new_I.append(2)
    elif (i=='50K -- 100K'):
        new_I.append(3)
    else:
        new_I.append(4)
new_I=pd.DataFrame(new_I)
test.insert(6,'New_Income',new_I)

train=train.drop( 'Income',axis=1)
vali=vali.drop( 'Income',axis=1)
test=test.drop('Income', axis=1)


new_E=[]

for i in train['Educational_Level']:
    
    if (i=='Mid-School'):
        new_E.append(1)
    elif (i=='High-School'):
        new_E.append(2)
    elif (i=='College'):
        new_E.append(3)
    else:
        new_E.append(4)

    
new_E=pd.DataFrame(new_E)
train.insert(5,'New_Education',new_E)



new_E=[]

for i in vali['Educational_Level']:
    
    if (i=='Mid-School'):
        new_E.append(1)
    elif (i=='High-School'):
        new_E.append(2)
    elif (i=='College'):
        new_E.append(3)
    else:
        new_E.append(4)

    
new_E=pd.DataFrame(new_E)
vali.insert(5,'New_Education',new_E)


new_E=[]

for i in test['Educational_Level']:
    
    if (i=='Mid-School'):
        new_E.append(1)
    elif (i=='High-School'):
        new_E.append(2)
    elif (i=='College'):
        new_E.append(3)
    else:
        new_E.append(4)

    
new_E=pd.DataFrame(new_E)
test.insert(5,'New_Education',new_E)

train=train.drop( 'Educational_Level',axis=1)
vali=vali.drop( 'Educational_Level',axis=1)
test=test.drop('Educational_Level',axis=1)


Reservation=[]

for i in train['Reservation_Status']:
    
    if (i=='Check-In'):
        Reservation.append(1)
    elif (i=='Canceled'):
        Reservation.append(2)
    elif (i=='No-Show'):
        Reservation.append(3)

    
Reservation=pd.DataFrame(Reservation)
train.insert(18,'New_Reservation_Status',Reservation)



Reservation=[]

for i in vali['Reservation_Status']:
    
    if (i=='Check-In'):
        Reservation.append(1)
    elif (i=='Canceled'):
        Reservation.append(2)
    elif (i=='No-Show'):
        Reservation.append(3)
  

    
Reservation=pd.DataFrame(Reservation)
vali.insert(18,'New_Reservation_Status',Reservation)

In [ ]:
train=train.drop( 'Reservation_Status',axis=1)
vali=vali.drop( 'Reservation_Status',axis=1)


target = 'New_Reservation_Status'
ID = 'Reservation-id'

y=train[target]
vali_target=vali[target]


In [ ]:
train=train.drop([target,ID,'Booking_date','Expected_checkin', 'Expected_checkout'],axis=1)
vali=vali.drop([target,ID,'Booking_date','Expected_checkin', 'Expected_checkout'],axis=1)
test=test.drop([ID,'Booking_date','Expected_checkin', 'Expected_checkout'],axis=1)


family=train[['Adults','Children','Babies']].sum(axis=1)
train.insert(6,'Family',family)

family=vali[['Adults','Children','Babies']].sum(axis=1)
vali.insert(6,'Family',family)

family=test[['Adults','Children','Babies']].sum(axis=1)
test.insert(6,'Family',family)

train=train.drop(['Adults','Children','Babies'],axis=1)
vali=vali.drop(['Adults','Children','Babies'],axis=1)
test=test.drop(['Adults','Children','Babies'],axis=1)

In [ ]:
numerical_column=[col for col in train.columns if train[col].dtype in ['int64','float64']]

# find the categorical column in the data
categorical_column=list(set(train.columns)-set(numerical_column))

print("Numerical columns",numerical_column)
print('Categorical columns',categorical_column)

In [ ]:
encoding_instance = OneHotEncoder(handle_unknown='ignore')
train_OneHot = pd.DataFrame(encoding_instance.fit_transform(train[categorical_column]).toarray())

encoding_instance = OneHotEncoder(handle_unknown='ignore')
vali_OneHot = pd.DataFrame(encoding_instance.fit_transform(vali[categorical_column]).toarray())

encoding_instance = OneHotEncoder(handle_unknown='ignore')
test_OneHot = pd.DataFrame(encoding_instance.fit_transform(test[categorical_column]).toarray())


train_data = pd.concat([train,train_OneHot],axis=1)
vali_data = pd.concat([vali,vali_OneHot],axis=1)
test_data=pd.concat([test,test_OneHot], axis=1)

In [ ]:
X = train_data.drop( categorical_column,axis=1)
scaler = StandardScaler().fit(X)
X = pd.DataFrame(scaler.transform(X))


vali = vali_data.drop(categorical_column,axis=1)
scaler = StandardScaler().fit(vali)
vali = pd.DataFrame(scaler.transform(vali))


test=test_data.drop(categorical_column,axis=1)
scaler=StandardScaler().fit(test)
test=pd.DataFrame(scaler.transform(test))

In [ ]:
def CreateBalancedSampleWeights(y_train, largest_class_weight_coef):
    classes = y_train.unique()
    classes.sort()
    class_samples = np.bincount(y_train)
    total_samples = class_samples.sum()
    n_classes = len(class_samples)
    weights = total_samples / (n_classes * class_samples * 1.0)
    class_weight_dict = {key : value for (key, value) in zip(classes, weights)}
    class_weight_dict[classes[1]] = class_weight_dict[classes[1]] * largest_class_weight_coef
    sample_weights = [class_weight_dict[y] for y in y_train]

    return sample_weights



largest_class_weight_coef = max(y.value_counts().values)/y.shape[0]
 #pass y_train as numpy array
weight = CreateBalancedSampleWeights(y, largest_class_weight_coef)

In [ ]:
from sklearn.utils.class_weight import compute_sample_weight
from xgboost import XGBRFClassifier
w=compute_sample_weight(class_weight='balanced', y=y)
#xg=XGBClassifier(n_estimators=200,learning_rate=0.05, max_depth=10)

#xg = XGBClassifier(objective="multi:softprob", n_estimators=600,learning_rate=0.01, max_depth=10,random_state=42)

xg = XGBRFClassifier(n_estimators=1000,learning_rate=0.01 ,max_depth=10,random_state=42)
xg.fit(X, y,sample_weight = w)
pred = xg.predict(vali)    
print ('-------------------------------------')
print('')
print(confusion_matrix(vali_target,pred))

a=f1_score(vali_target,pred, average='macro')

print(a)

In [ ]:
sss

In [ ]:

test_prediction=xg.predict(test)
output = pd.DataFrame({'Reservation-id': test_set[ID],
                  'Reservation_Status': test_prediction})
output.to_csv('my_submission.csv', index=False)